In [1]:
import os

In [ ]:
%pwd

In [2]:
os.chdir("../")

In [ ]:
%pwd

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [4]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(self,config_path = CONFIG_FILE_PATH,params_path = PARAMS_FILE_PATH,schema_path = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        self.schema = read_yaml(schema_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir= config.unzip_dir
        )

        return data_ingestion_config



In [6]:
import os
import urllib.request as request
import zipfile
from src.mlProject import logger
from src.mlProject.utils.common import get_size

In [7]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig) -> None:
       self.config = config
    

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url = self.config.source_URL,
                filename=self.config.local_data_file

            )
            logger.info(f"{filename} download ! with following info: \n{headers}")
        else:
            logger.info(f"file already exist of size :{get_size(Path(self.config.local_data_file))}")
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [8]:
try:
    config = ConfigurationManager()
    dataingestion_config = config.get_data_ingestion()
    dataingestion = DataIngestion(dataingestion_config)
    dataingestion.download_file()
    dataingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-02-13 19:47:36,819:INFO:common:yaml file:config\config.yaml loaded successfully]
[2024-02-13 19:47:36,829:INFO:common:yaml file:params.yaml loaded successfully]
[2024-02-13 19:47:36,840:INFO:common:yaml file:schema.yaml loaded successfully]
[2024-02-13 19:47:36,842:INFO:common:created directory at: artifacts]


[2024-02-13 19:47:37,175:INFO:common:created directory at: artifacts/data_ingestion]
[2024-02-13 19:47:38,814:INFO:2865788197:artifacts/data_ingestion/data.zip download ! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: C986:667F:1269D6:15D939:65CB7A07
Accept-Ranges: bytes
Date: Tue, 13 Feb 2024 14:17:44 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10224-MAA
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1707833864.354282,VS0,VE306
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: b0492f7d2d4c4491616135909cd115d21358981f
Expires: Tue, 13 Feb